In [16]:
from matplotlib import pylab
import nengo
import random
import numpy as np
import gzip as gz
from cPickle import load
try:
    import Image
except ImportError:
    from PIL import Image
from scipy.sparse.linalg import svds
import scipy
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.animation as animation

#%matplotlib inline #Makes visualizations appar inline (Commented out because animation popup as new window)

In [17]:
def load_img(path, dims):
    """Load the image at path and return an array representing the raster.
    Flattens image. Shifts pixel activations such that 0 represents gray,
    normalizes the output array.
    Keyword arguments:
    path -- str, path of the image to be loaded.
    dims -- (w, h), where w,h are ints indicating dimensions of the image (in
        px)."""

    img = Image.open(path).resize(dims).getdata()
    img.convert('L')
    img = subtract(array(img).flatten(), 127.5)
    return img/norm(img)


def load_data(filename):
    """Uncompress, unpickle and return a .pkl.gz file.
    Keyword arguments:
    filename -- str, a valid file path"""

    return load(gz.open(filename))

def load_mini_mnist(option=None):
    """Load and return the first \%10 of the images in the mnist dataset.
    Does not return labels. Pass in 'train', 'valid' or 'test' if you want to
    load a specific subset of the dataset.
    Keyword arguments:
    option -- str (default=None)."""

    mini_mnist = load(gz.open('./mini_mnist.pkl.gz', 'rb'))
    if option == 'train':
        return mini_mnist[0]
    elif option == 'valid':
        return mini_mnist[1]
    elif option == 'test':
        return mini_mnist[2]
    else:
        return mini_mnist

In [18]:
def rotate_img(img, degrees):
    '''Rotates image the degrees passed in counterclockwise
    Reshapes image to original shape
    '''
    original = img.shape
    
    newImg = scipy.ndimage.interpolation.rotate(np.reshape(img, (dim,dim), 'F'),degrees,reshape=False)
    newImg = np.reshape(newImg, original, 'F')
    return newImg

In [19]:
conn_synapse = 0.1 #post synaptic time constant to use for filtering (pstc) - what does changing this do?
probe_synapse = 0.01 #pstc
multiplier = 2 #not used
n_neurons = 5000
direct = True #Direct - function computed explicitly instead of in neurons 
stop_time = 3.0
run_time = 3.0 #in seconds

In [20]:
dim = 28 #size of the image
mnist = load_mini_mnist()
train = mnist[0] #collection of training images
img = mnist[1][0] #image to be used for testing
compress_size = 400 #?
basis, S, V = svds(train.T, k=compress_size) #Used for encoding and decoding information 
    #a set of vectors representing what a hand drawn number should look like?

In [21]:
#Need same number of vectors in basis as number of neurons (randomly sample from basis)
expanded_basis = np.array([random.choice(basis.T) for _ in range(n_neurons)]) 

In [22]:
def stim_func(t):
    '''returns the image for first 0.1s'''
    if t < 0.1:
        return img
    else:
        return [0 for _ in range(len(img))]

In [23]:
def stim_func_rot(t):
    if t < 0.2:
        return 0
    elif t<0.4:
        return 10
    else:
        return 1

In [24]:
def connection_func(x):
    '''takes the output from the first ensemble and rotates it degress specified by stim''' 
    return rotate_img(x[:-1],x[-1])
   

In [10]:
# Interference from stimulus when trying to rotate, so tried to delay rotation function

def node_func(t,x):
    '''takes the output from the first ensemble and rotates it 1 degrees'''
    if t < 0.1:
        return x
    else:
        #return rotate_img(x,10) - (x*0.1)
        return rotate_img(x,1)

In [26]:
#Sprite?
with nengo.Network() as net:
    
    if direct:
        neuron_type = nengo.Direct() #function computed explicitly, instead of in neurons
    else:
        neuron_type = nengo.LIF() #spiking version of the leaky integrate-and-fire neuron model

    #Input stimulus - provide data to the ensemble
    ipt = nengo.Node(stim_func)
    
    ipt2 = nengo.Node(stim_func_rot)
    
    #Group of neurons that collectively represent information(vector)
    ens = nengo.Ensemble(n_neurons,
                         dimensions=dim**2+1, #pixels of the image? 28*28
                         encoders=expanded_basis, #transform representational space to neuron space
                         eval_points=expanded_basis, #used for decoder solving, spanning interval
                         n_eval_points=n_neurons,
                         neuron_type=neuron_type)

    nengo.Connection(ipt, #source nengo object
                     ens[:-1], #destination object
                     synapse=None, #pstc
                     transform=1) #linear transformation, what does changing this do?
    
    nengo.Connection(ipt2,ens[-1],synapse=None,transform =1)
    
    '''Connection has param solver - (tried to make weight stronger, for image to last longer)
    solver : Solver, optional (Default: nengo.solvers.LstsqL2())
    Solver instance to compute decoders or weights (see Solver). If solver.weights is True, a full 
    connection weight matrix is computed instead of decoders.'''
    #s = nengo.solvers.LstsqL2(weights=True)
    
    conn = nengo.Connection(ens, ens, synapse=conn_synapse,transform=1, function=connection_func)
    
    #delaynode = nengo.Node(node_func, size_in = dim**2, size_out =dim**2) #does processing on information
    #conn = nengo.Connection(ens, delaynode, synapse=conn_synapse,transform =1) #incr transform, rotates faster
    #conn2 = nengo.Connection(delaynode, ens, synapse=conn_synapse, transform =1)
    
    probe = nengo.Probe(ens, attr='decoded_output',#sample_every=0.001,
                       synapse=probe_synapse)
   

ValueError: Validation error when setting 'Ensemble.encoders': shape[1] should be 785 (got 784)

In [12]:
sim = nengo.Simulator(net)

In [13]:
sim.run(run_time)

Simulation finished in 0:00:04.                                                 


In [14]:
pylab.imshow(np.reshape(img, (dim,dim), 'F'), cmap='Greys_r')

In [15]:
'''Image at stop time'''
pylab.imshow(np.reshape([0. if x < 0.00001 else x for x in sim.data[probe][int(stop_time*1000)-1]], 
                             (dim, dim), 'F'), cmap=plt.get_cmap('Greys_r'),animated=True)

In [16]:
'''Image at start time'''
pylab.imshow(np.reshape([0. if x < 0.00001 else x for x in sim.data[probe][1]], 
                             (dim, dim), 'F'), cmap=plt.get_cmap('Greys_r'),animated=True)

In [17]:
'''Animation for Probe output'''
fig = plt.figure()

def updatefig(i):
    im = pylab.imshow(np.reshape([0. if x < 0.00001 else x for x in sim.data[probe][i]],
                                 (dim, dim), 'F'), cmap=plt.get_cmap('Greys_r'),animated=True)
    
    return im,

ani = animation.FuncAnimation(fig, updatefig, interval=1, blit=True)
plt.show()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Python27\lib\lib-tk\Tkinter.py", line 1536, in __call__
    return self.func(*args)
  File "C:\Python27\lib\lib-tk\Tkinter.py", line 587, in callit
    func(*args)
  File "C:\Python27\lib\site-packages\matplotlib\backends\backend_tkagg.py", line 147, in _on_timer
    TimerBase._on_timer(self)
  File "C:\Python27\lib\site-packages\matplotlib\backend_bases.py", line 1305, in _on_timer
    ret = func(*args, **kwargs)
  File "C:\Python27\lib\site-packages\matplotlib\animation.py", line 1021, in _step
    still_going = Animation._step(self, *args)
  File "C:\Python27\lib\site-packages\matplotlib\animation.py", line 827, in _step
    self._draw_next_frame(framedata, self._blit)
  File "C:\Python27\lib\site-packages\matplotlib\animation.py", line 845, in _draw_next_frame
    self._pre_draw(framedata, blit)
  File "C:\Python27\lib\site-packages\matplotlib\animation.py", line 858, in _pre_draw
    self._blit_clear(self.